# Replacement Score Test

## Load the model from the local checkpoint (save from HF before)

In [1]:
import pathlib

from crosslayer_transcoder.utils.model_converters.circuit_tracer import (
    CircuitTracerConverter,
)
from transformer_lens.loading_from_pretrained import get_pretrained_model_config
import torch

In [2]:
from crosslayer_transcoder.utils.module_builder import build_module_from_config, yaml_to_config
from crosslayer_transcoder.utils.checkpoints import load_model_from_lightning_checkpoint

config = yaml_to_config("../../config/circuit-tracer.yaml")
clt_module = build_module_from_config(config)

# load checkpoint
checkpoint = "../checkpoints/clt.ckpt"

clt_module = load_model_from_lightning_checkpoint(clt_module, checkpoint)


save_dir = pathlib.Path("clt_module_test")
feature_input_hook = "hook_resid_mid"
feature_output_hook = "hook_mlp_out"

converter = CircuitTracerConverter(
save_dir=save_dir,
feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
)
converter.convert_and_save(clt_module, dtype=torch.bfloat16) 

model {'class_path': 'crosslayer_transcoder.model.clt.CrossLayerTranscoder', 'init_args': {'encoder': {'class_path': 'crosslayer_transcoder.model.clt.Encoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_layers': 12}}, 'decoder': {'class_path': 'crosslayer_transcoder.model.clt.CrosslayerDecoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_layers': 12}}, 'nonlinearity': {'class_path': 'crosslayer_transcoder.model.jumprelu.JumpReLU', 'init_args': {'theta': 0.03, 'bandwidth': 0.01, 'n_layers': 12, 'd_features': 10000}}, 'input_standardizer': {'class_path': 'crosslayer_transcoder.model.standardize.DimensionwiseInputStandardizer', 'init_args': {'n_layers': 12, 'activation_dim': 768}}, 'output_standardizer': {'class_path': 'crosslayer_transcoder.model.standardize.DimensionwiseOutputStandardizer', 'init_args': {'n_layers': 12, 'activation_dim': 768}}}}
encoder {'class_path': 'crosslayer_transcoder.model.clt.Encoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_laye

Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1561 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1174 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2459 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2027 > 1024). Running this sequence through the model will result in indexing errors


dead_features {'class_path': 'crosslayer_transcoder.metrics.dead_features.DeadFeatures', 'init_args': {'n_features': 10000, 'n_layers': 12, 'return_per_layer': True, 'return_log_freqs': True, 'return_neuron_indices': True}}
n_features 10000
n_layers 12
return_per_layer True
return_log_freqs True
return_neuron_indices True
learning_rate 3e-4
compile True
lr_decay_step 16000
lr_decay_factor 0.1
lambda_sparsity 0.0007
c_sparsity 1
use_tanh True
pre_actv_loss 1e-6
compute_dead_features True
compute_dead_features_every 500
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])
JumpReLU()
torch.Size([1, 12, 10000])


In [3]:
from circuit_tracer.transcoder.cross_layer_transcoder import load_clt
transcoder, state_dict_pre_load = load_clt(
    clt_path=save_dir.as_posix(),
    lazy_decoder=False,
    lazy_encoder=False,
    feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
    dtype=torch.bfloat16,
)


12
['W_enc_0', 'b_dec_0', 'b_enc_0', 'threshold_0']
['W_enc_1', 'b_dec_1', 'b_enc_1', 'threshold_1']
['W_enc_2', 'b_dec_2', 'b_enc_2', 'threshold_2']
['W_enc_3', 'b_dec_3', 'b_enc_3', 'threshold_3']
['W_enc_4', 'b_dec_4', 'b_enc_4', 'threshold_4']
['W_enc_5', 'b_dec_5', 'b_enc_5', 'threshold_5']
['W_enc_6', 'b_dec_6', 'b_enc_6', 'threshold_6']
['W_enc_7', 'b_dec_7', 'b_enc_7', 'threshold_7']
['W_enc_8', 'b_dec_8', 'b_enc_8', 'threshold_8']
['W_enc_9', 'b_dec_9', 'b_enc_9', 'threshold_9']
['W_enc_10', 'b_dec_10', 'b_enc_10', 'threshold_10']
['W_enc_11', 'b_dec_11', 'b_enc_11', 'threshold_11']
dict_keys(['b_dec', 'b_enc', 'activation_function.threshold', 'W_enc', 'W_dec.0', 'W_dec.1', 'W_dec.2', 'W_dec.3', 'W_dec.4', 'W_dec.5', 'W_dec.6', 'W_dec.7', 'W_dec.8', 'W_dec.9', 'W_dec.10', 'W_dec.11'])
JumpReLU
W_enc
torch.Size([12, 10000, 768])


In [4]:
import einops
from safetensors import safe_open
import torch
import os
# sanity check against original model 

print(transcoder.clt_path)
clt_path = save_dir.as_posix()
assert transcoder.clt_path == clt_path

# for i in range(clt_module.model.encoder.n_layers):
#     # THESE SHOULD NOT be close
#     assert not torch.allclose(clt_module.model.encoder.W[i].T.to('cuda:0'), transcoder.W_enc[i].to('cuda:0')) # should fail bc of folding 

# TEST: state_dict_pre_load weights match the files before being loaded into the model
for i in range(transcoder.n_layers):
    enc_file = os.path.join(clt_path, f"W_enc_{i}.safetensors")
    with safe_open(enc_file, framework="pt", device=transcoder.device.type) as f:
        w_file = f.get_tensor(f"W_enc_{i}").to(
            dtype=state_dict_pre_load["W_enc"][i].dtype, device=state_dict_pre_load["W_enc"][i].device
        )
        w_state = state_dict_pre_load["W_enc"][i]
        assert w_file.shape == w_state.shape, f"W_enc_{i} shape mismatch: {w_file.shape} != {w_state.shape}"
        assert w_file.dtype == w_state.dtype, f"W_enc_{i} dtype mismatch: {w_file.dtype} != {w_state.dtype}"
        assert torch.allclose(w_file, w_state), (
            i, (w_file - w_state).abs().max().item()
        )


# TEST: weights in the files should equal weights from the transcoder
for i in range(transcoder.n_layers):
    w_model = transcoder._get_encoder_weights(i)  # works for both lazy and eager
    enc_file = os.path.join(clt_path, f"W_enc_{i}.safetensors")
    with safe_open(enc_file, framework="pt", device=transcoder.device.type) as f:
        w_file = f.get_tensor(f"W_enc_{i}").to(
            dtype=w_model.dtype, device=w_model.device
        )

    assert w_model.shape == w_file.shape
    assert w_model.dtype == w_file.dtype
    assert torch.allclose(w_model, w_file), (i, (w_model - w_file).abs().max().item())


# fold
standardizer = clt_module.model.input_standardizer
W_enc_folded, b_enc_folded = standardizer.fold_in_encoder(clt_module.model.encoder.W.to(dtype=torch.bfloat16), clt_module.model.encoder.b.to(dtype=torch.bfloat16))

W_enc_folded = W_enc_folded.to(dtype=torch.bfloat16)
b_enc_folded = b_enc_folded.to(dtype=torch.bfloat16)

state_dict = {}
device = clt_module.device 
# TEST: eights in the file should equal the folded weights
for i in range(clt_module.model.encoder.n_layers):
    enc_file = f"W_enc_{i}.safetensors"
    with safe_open(os.path.join(clt_path, enc_file), framework="pt", device=device.type) as f:
        assert W_enc_folded[i].T.shape == f.get_tensor(f"W_enc_{i}").shape, f"W_enc_{i} shape mismatch: {W_enc_folded[i].shape} != {f.get_tensor(f'W_enc_{i}').shape}"
        assert W_enc_folded[i].dtype == f.get_tensor(f"W_enc_{i}").dtype, f"W_enc_{i} dtype mismatch: {W_enc_folded[i].dtype} != {f.get_tensor(f'W_enc_{i}').dtype}"
        assert torch.allclose(W_enc_folded[i].T, f.get_tensor(f"W_enc_{i}"))
        assert torch.allclose(b_enc_folded[i], f.get_tensor(f"b_enc_{i}"))


        # loaded model 
        assert transcoder.W_enc[i].shape == f.get_tensor(f"W_enc_{i}").shape
        assert transcoder.W_enc[i].dtype == f.get_tensor(f"W_enc_{i}").dtype, f"W_enc_{i} dtype mismatch: {transcoder.W_enc[i].dtype} != {f.get_tensor(f'W_enc_{i}').dtype}"
        assert torch.allclose(transcoder.W_enc[i], f.get_tensor(f"W_enc_{i}").to("cuda:0"))


    


for i in range(clt_module.model.encoder.n_layers):
    rearranged_W_enc = einops.rearrange(
        W_enc_folded[i],
        "d_acts d_features -> d_features d_acts",
    ).contiguous()
    assert torch.allclose(rearranged_W_enc.to('cuda:0'), transcoder.W_enc[i].to('cuda:0'))
    assert torch.allclose(b_enc_folded[i].to(dtype=torch.bfloat16).to('cuda:0'),transcoder.b_enc[i].to(dtype=torch.bfloat16).to('cuda:0'))

clt_module_test


In [5]:
from circuit_tracer import ReplacementModel


config = get_pretrained_model_config("gpt2")
config.d_type = torch.float16
rm = ReplacementModel.from_config(
    config=config,
    transcoders=transcoder,
)


print(rm.transcoders)
print(rm.transcoders.W_enc.shape)
assert torch.allclose(rm.transcoders.W_enc, transcoder.W_enc)
print([x.shape for x in list(rm.transcoders.parameters())])
print(rm.transcoders.activation_function)

## TODO: sanity check model



CrossLayerTranscoder(
  (activation_function): JumpReLU(
    threshold=Parameter containing:
    tensor([[[0.2344, 0.2080, 0.2246,  ..., 0.2207, 0.2363, 0.2314]],
    
            [[0.2451, 0.2334, 0.2578,  ..., 0.2676, 0.2432, 0.2256]],
    
            [[0.2402, 0.2363, 0.2402,  ..., 0.2139, 0.2520, 0.2002]],
    
            ...,
    
            [[0.2168, 0.3789, 0.2480,  ..., 0.2305, 0.3242, 0.2793]],
    
            [[0.2891, 0.2158, 0.1943,  ..., 0.2793, 0.2812, 0.2949]],
    
            [[0.1934, 0.1543, 0.1758,  ..., 0.2832, 0.2324, 0.2197]]],
           device='cuda:0'), bandwidth=2
  )
  (W_dec): ParameterList(
      (0): Parameter containing: [torch.float32 of size 10000x12x768 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 10000x11x768 (cuda:0)]
      (2): Parameter containing: [torch.float32 of size 10000x10x768 (cuda:0)]
      (3): Parameter containing: [torch.float32 of size 10000x9x768 (cuda:0)]
      (4): Parameter containing: [torch.float32 of si

In [6]:
prompt = (
    "The capital of state containing Dallas is"  # What you want to get the graph for
)
max_n_logits = 10  # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 100  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size = 256//8  # Batch size when attributing
offload = (
    "cpu"
)  # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report


In [7]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils import create_graph_files

torch.cuda.empty_cache()

# FOr some reason this takes up a lot of VRAM
graph = attribute(
    prompt=prompt,
    model=rm,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose
)

Phase 0: Precomputing activations and vectors


Tokens shape: torch.Size([8])
W_enc_layer shape: torch.Size([10000, 768])
sparse_W_enc_layer nnz: 7680000
layer_features shape: torch.Size([8, 10000])
sparse_pre_activations nnz: 80000
layer_features shape after activation: torch.Size([8, 10000])
l0: 70000
sparse_layer nnz: 70000
W_enc_layer shape: torch.Size([10000, 768])
sparse_W_enc_layer nnz: 7680000
layer_features shape: torch.Size([8, 10000])
sparse_pre_activations nnz: 80000
layer_features shape after activation: torch.Size([8, 10000])
l0: 70000
sparse_layer nnz: 70000
W_enc_layer shape: torch.Size([10000, 768])
sparse_W_enc_layer nnz: 7680000
layer_features shape: torch.Size([8, 10000])
sparse_pre_activations nnz: 80000
layer_features shape after activation: torch.Size([8, 10000])
l0: 70000
sparse_layer nnz: 70000
W_enc_layer shape: torch.Size([10000, 768])
sparse_W_enc_layer nnz: 7680000
layer_features shape: torch.Size([8, 10000])
sparse_pre_activations nnz: 80000
layer_features shape after activation: torch.Size([8, 10000])


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.62 GiB. GPU 0 has a total capacity of 44.34 GiB of which 10.35 GiB is free. Process 2491674 has 8.54 GiB memory in use. Process 2496736 has 25.44 GiB memory in use. Of the allocated memory 23.23 GiB is allocated by PyTorch, and 1.91 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
graph_dir = 'graphs'
graph_name = 'example_graph.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

graph.to_pt(graph_path)